In [46]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import utils
from whittaker_smooth import whittaker_smooth
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [47]:
IMGS_PATH = '/home/mauricio/code/Plant-Growth-Cycle-based-on-geospatial-data/Manitoba1-1/'
bands =  ['B4','B3','B2','B8']

# Dict of the images.
images = utils.load_landsat_image(IMGS_PATH, bands)

# Get the days after plant and the dates.
days, dates, hours = utils.images_time_info(images.keys())

# Calculate the NDVI series over the crop.
ndvi_series = utils.generate_ndvi_time_series(images)

IndexError: list index out of range

In [ ]:
plt.plot(days, ndvi_series)
plt.title('NDVI Manitoba 2022')
plt.show()

In [ ]:
# Perform a Whittaker Smooth.
y_ = whittaker_smooth(np.array(ndvi_series), lmbd=10)
plt.plot(days, ndvi_series, label="Raw")
plt.plot(days, y_, label='Wittaker')
plt.legend()
plt.show()

In [ ]:
df = pd.DataFrame(columns=["Dates", "Values"])
df["Dates"] = pd.to_datetime(dates)
df["Values"] = y_
df = df.resample('1D', on="Dates").agg("first").ffill() ## Quitar para eliminar resampling
df

In [3]:
df.plot()

NameError: name 'df' is not defined

## Train/Test

In [ ]:
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = pd.concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]

# fit an xgboost model and make a one step prediction
def xgboost_forecast(train, testX):
	# transform list into array
	train = np.asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict(np.asarray([testX]))
	return yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1], test[i, -1]
		# fit model on history and make a prediction
		yhat = xgboost_forecast(history, testX)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
		# summarize progress
		print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
	error = mean_absolute_error(test[:, -1], predictions)
	return error, test[:, -1], predictions

In [ ]:
# load the dataset
values = list(df["Values"])
# transform the time series data into supervised learning
data = series_to_supervised(values, n_in=6)
# evaluate
mae, y, yhat = walk_forward_validation(data, 12)
print('MAE: %.3f' % mae)
# plot expected vs preducted
plt.plot(y, label='Expected')
plt.plot(yhat, label='Predicted')
plt.legend()
plt.show()

In [ ]:
y_

## Forecast

In [ ]:
# load the dataset
values = list(df["Values"])
# transform the time series data into supervised learning
train = series_to_supervised(values, n_in=90)
# split into input and output columns
trainX, trainy = train[:, :-1], train[:, -1]
# fit model
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
model.fit(trainX, trainy)

predictions = []

for i in range(300):
# construct an input for a new preduction
    row = values[-90:]
# make a one-step prediction
    yhat = model.predict(np.asarray([row]))
    values.append(yhat[0])
    predictions.append(yhat[0])
    #print('Input: %s, Predicted: %.3f' % (row, yhat[0]))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot([i for i in range(len(values))], values, label="Previous")
plt.plot([i for i in range(len(df["Values"]), len(df["Values"])+len(predictions))], predictions, label="Predictions")
plt.legend()
plt.show()

# Interpolation

In [ ]:
IMGS_PATH = '/home/mauricio/code/Plant-Growth-Cycle-based-on-geospatial-data/Manitoba1-1/'
bands =  ['B4','B3','B2','B8']

# Dict of the images.
images = utils.load_landsat_image(IMGS_PATH, bands)

# Get the days after plant and the dates.
days, dates, hours = utils.images_time_info(images.keys())

# Calculate the NDVI series over the crop.
ndvi_series = utils.generate_ndvi_time_series(images)

In [ ]:
plt.plot(days, ndvi_series)
plt.title('NDVI Manitoba 2022')
plt.show()

In [ ]:
# Perform a Whittaker Smooth.
smoothed = whittaker_smooth(np.array(ndvi_series), lmbd=10)
plt.plot(days, ndvi_series, label="Raw")
plt.plot(days, smoothed, label='Wittaker')
plt.legend()
plt.show()

In [ ]:
plt.plot(days)

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(x_new, y_new, 'b', label='Interpolated')
plt.plot(days, smoothed, 'ro', label='smooth curve')
plt.title('Cubic Spline Interpolation')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [69]:
# load the dataset
values = list(y_new)
# transform the time series data into supervised learning
train = series_to_supervised(values, n_in=60)
# split into input and output columns
trainX, trainy = train[:, :-1], train[:, -1]
# fit model
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
model.fit(trainX, trainy)

predictions = []

for i in range(1000):
# construct an input for a new preduction
    row = values[-60:]
# make a one-step prediction
    yhat = model.predict(np.asarray([row]))
    values.append(yhat[0])
    predictions.append(yhat[0])
    #print('Input: %s, Predicted: %.3f' % (row, yhat[0]))